# Урок 2. Наивный байесовский классификатор.

Наивный байесовский классификатор - семейство алгоритмов классификации, которые принимают допущение о том, что каждый параметр классифицируемых данных не зависит от других параметров объектов, т.е. ни один из параметров не оказывает влияние на другой.
Согласитесь, что достаточно наивно было бы предполагать, что, допустим, рост и вес человека - совершенно независимые параметры.

Для начала посмотрим на примере, как работает этот алгоритм.
Допустим, у нас имеется небольшой набор данных Fruits, в котором представлена информация о видах {banana, orange, plum}. Отметим, что это просто конкретные имеющиеся "измерения", на которых обучается модель, и мы хотим научиться определять по этим данным, какой фрукт перед нами.

В последнем столбце $Total$ представлено количество фруктов определенного класса (500 бананов, 300 апельсинов и 200 слив - всего 1000). В строке же $Total$ - общее количество фруктов с определенными признаками {long, sweet, yellow}.

In [2]:
import numpy as np
import pandas as pd

data = np.array([[400, 350, 450, 500],
                 [0, 150, 300, 300],
                 [30, 180, 100, 200],
                 [430, 680, 850, 1000]])
idx = ['Banana', 'Orange', 'Plum', 'Total']
col = ['Long', 'Sweet', 'Yellow', 'Total']

fruits = pd.DataFrame(data, columns=col, index=idx)
fruits

,Long,Sweet,Yellow,Total
Banana,400,350,450,500
Orange,0,150,300,300
Plum,30,180,100,200
Total,430,680,850,1000


Здесь для простоты мы оперируем интуитивно понятными бинарными признаками, т.е. фактически любой из признаков мы можем определить как "да" или "нет". В соответствии с этим, например, получается, что среди бананов 400 из 500 длинные, 350 - сладкие, а 450 - желтые. Соответственно, если у нас "на руках" именно эти данные, то можно сказать, что, если у нас в руках банан, то он с вероятностью 80% длинный (вероятность считаем как отношение объектов с данным признаком к общему числу объектов -в данном случае 400/500 = 0,8), с вероятностью 70% (350/500 = 0,7) - сладкий, и с вероятностью 90% (450/500 = 0,9) - желтый.

В случае с дискретными признаками (которые принимают значения исключительно из определенного набора значений) всегда составляется подобная частотная таблица для каждого класса и каждого признака, указывающая, сколько раз значение определенного признака встречается в каждом классе.

Теперь, если мы получим только параметры - длину, сладость и цвет фрукта, сможем вычислить вероятность того, что фрукт является бананом, апельсином или сливой. Допустим, мы хотим узнать, к какому классу относится фрукт с параметрами {длинный, сладкий, желтый}. Вероятность того, что объект принадлежит какому-либо классу при данных параметрах обозначается как:

$$
P(Class|Long, Sweet, Yellow)
$$

Чтобы вычислить вероятность в случае дискретных (конечных) данных, мы делим количество соответствующих признаку объектов на общее число объектов. Например, если мы хотим узнать вероятность $P(Long|Banana)$, мы вычисляем $400 / 500 = 08$, как уже упоминалось выше.

Данный метод основан на теореме английского математика-статистика Томаса Байеса. По сути, она позволяет предсказать класс на основании набора параметров, используя вероятность. Общая формула Байеса для класса с одним признаком выглядит так:

$$
P(Class A|Feature 1) = \frac{P(Feature 1|Class A)\cdot P(Class A)}{P(Feature 1)}
$$

$P(Class A|Feature 1)$ - вероятность того, что объект является классом $A$ при том, что его признак соответствует $Feature 1$.

Упрощенное уравнение для классификации при двух признаках выглядит так:

$$
P(Class A|Feature 1, Feature 2) = \frac{P(Feature 1|Class A)\cdot P(Feature 2|Class A)\cdot P(Class A)}{P(Feature 1)\cdot P(Feature 2)}
$$

И далее для большего количества признаков формула меняется соответствующим образом.

Напомним, что вероятность - число, которое принимает значения от 0 до 1, при этом 0 - полное несоответствие класса признакам, а 1 - однозначно определенный класс. Соответственно, чем ближе значение вероятности определенного класса к 1, тем больше шанс того, что объект принадлежит именно этому классу.

Знаменатель можно проигнорировать, так как он будет одинаков для всех вычислений и никакой важной информации не даст.
Тогда получится следующее уравнение:

$$
P(Class A|Feature 1, Feature 2) = P(Feature 1|Class A)\cdot P(Feature 2|Class A)\cdot P(Class A)
$$

То есть для каждого возможного класса вычисляем только произведение вероятностей того, что каждый признак соответствует классу, и вероятности того, что объект принадлежит этому классу. Соответственно, наибольшее значение этого произведения, рассчитанного с признаками конкретного объекта, для какого-то из классов будет указывать на принадлежность объекта к этому классу.

Вернемся к нашему примеру с фруктами. Чтобы понять, к какому классу принадлежит объект с признаками {Long, Sweet, Yellow}, рассчитаем для каждого из классов формулу Байеса, используя для этого данные частотной таблицы. На выходе получаем вероятность того, что объект принадлежит классу. Класс с наибольшей вероятностью является ответом.

In [3]:
result = {}
for i in range (fruits.values.shape[0] - 1):
    p = 1
    for j in range (fruits.values.shape[1] - 1):
        p *= fruits.values[i, j] / fruits.values[i, -1]
    p *= fruits.values[i, -1] / fruits.values[-1, -1]
    result[fruits.index[i]] = p

result

{'Banana': 0.252, 'Orange': 0.0, 'Plum': 0.013500000000000002}

Как мы видим, фрукт с параметрами {long, sweet, yellow} с наибольшей вероятностью принадлежит классу "Banana", что, кажется, соответствует реальности.

Но что делать, если у нас не частотная таблица, как в примере выше, а непрерывные данные? Например, мы не можем вычислять, сколько человек с конкретным ростом 1,81м, 1,67м и т.д. присутствует в выборке - нам это попросту ничего не даст, а лишь добавит громоздких вычислений. Поэтому обычно при непрерывных значениях параметров используется гауссовский наивный Байес, в котором сделано предположение о том, что значения параметров взяты из нормального распределения.

![](https://248006.selcdn.ru/public/DS_Block2_M5_final/GNB.gif)

На графике - плотность вероятности нормального распределения. По сути, где больше площадь под графиком, там и наиболее вероятные значения. Поскольку способ представления значений в наборе данных изменяется, то и формула условной вероятности изменяется на

$$
p(x_i|y) = \frac{1}{\sqrt{2\pi \sigma^2_y}}exp(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y})
$$

Здесь $\sigma^2$ - дисперсия (разброс) данных, а $\mu$ - математическое ожидание (среднее значение). При этом $y$ - предполагаемый класс, а $x_i$ - значение признака у того объекта, который нужно классифицировать.

По большому счету в нашей начальной формуле для вычисления вероятности того, что объект с данными признаками относится к конкретному классу, мы просто заменяем формулу вычисления вероятности как отношения количества соответствующих признаку объектов к общему числу объектов на данную, а дальше проводим идентичные вычисления.

Помимо гауссовского существуют еще два типа моделей на основе наивного байесовского классификатора: полиномиальная и с распределением Бернулли. *Полиномиальная* в основном используется для задачи классификации документов, т.е. определяет, к каким категориям относится текст: спорт, политика, техника и т.д. Используемые признаки являются частотой слов, присутствующих в документе. *Классификатор на основе распределения Бернулли* похож на полиномиальный метод, но признаками являются булевы переменные, т.е. они принимают только значения "yes" и "no" - например, встречается слово в тексте или нет. Используется в основном для классификации небольших текстов.

Теперь импортируем модель GaussianNB из библиотеки sklearn и посмотрим, как она работает на уже известном нам датасете Iris.

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris_dataset = load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris_dataset.data, iris_dataset.target, test_size=0.2, random_state=17)

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
nb = GaussianNB()

In [9]:
nb_model = nb.fit(x_train, y_train)

Получим предсказания для тестовых данных.

In [10]:
nb_predictions = nb.predict(x_test)
nb_predictions

array([0, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 0, 1, 0, 2, 0, 0, 2, 2, 2, 1, 0,
       2, 1, 1, 1, 1, 1, 0, 1])

Для определения точности предсказаний воспользуемся встроенной функцией *score*.

In [11]:
accuracy = nb.score(x_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9666666666666667


Вспомним результаты метода kNN.

In [12]:
from sklearn.neighbors import KNeighborsClassifier
k_n = KNeighborsClassifier(n_neighbors=3)
k_n_model = k_n.fit(x_train, y_train)
knn_predictions = k_n_model.predict(x_test)

In [15]:
accuracy = accuracy_score(y_test, knn_predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9666666666666667


Как видно, на одних и тех же данных алгоритм GaussianNB работает так же, как и kNN. Вполне возможно, что при корректировке количества соседей в kNN метод будет работать точнее, но при равных условиях подбор оптимальных гиперпараметров займет явно больше времени, чем использование алгоритма "из коробки".

Несмотря на простоту, наивный байесовский алгоритм может быть удивительно точным. В целом, наивные байесовские алгоритмы используются в анализе настроений, фильтрации спама, системах рекомендаций и т.д. Они быстры и просты в реализации, но их основным недостатком является требование к признакам быть независимыми. В большинстве реальных датасетов они являются зависимыми, а это в свою очередь снижает производительность классификатора.